In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import numpy as np
from sweep.experiment import Experiment, ExperimentType
from model.data import DataModel, KFeaturesDefinition, k_features_factory
from erm.problems.problems import ProblemType
import subprocess
import logging
logging.basicConfig(level=logging.DEBUG)


Define a datamodel

In [27]:
d = 100
vanilla_gaussian_diagonal = [(1, d)]
x_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
θ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ω_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
δ_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)
ν_diagonal = KFeaturesDefinition(diagonal=vanilla_gaussian_diagonal)

k_features_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}

data_model = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=k_features_kwargs,
)

Define an experiment

In [28]:
data_model

DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[(1, 100)]), 'θ_diagonal': KFeaturesDefinition(diagonal=[(1, 100)]), 'ω_diagonal': KFeaturesDefinition(diagonal=[(1, 100)]), 'δ_diagonal': KFeaturesDefinition(diagonal=[(1, 100)]), 'ν_diagonal': KFeaturesDefinition(diagonal=[(1, 100)])}, name='', description='', gamma=1)

In [29]:
from sweep.experiment import NumpyEncoder, NumpyDecoder
import json

data_model_json = json.dumps(data_model, cls=NumpyEncoder)

TypeError: Object of type DataModel is not JSON serializable

In [ ]:
data_model_json

'{"d": 100, "normalize_matrices": false, "data_model_factory_name": "k_features_factory", "factory_kwargs": {"x_diagonal": {"diagonal": [[1, 100]]}, "\\u03b8_diagonal": {"diagonal": [[1, 100]]}, "\\u03c9_diagonal": {"diagonal": [[1, 100]]}, "\\u03b4_diagonal": {"diagonal": [[1, 100]]}, "\\u03bd_diagonal": {"diagonal": [[1, 100]]}}, "name": "", "description": "", "gamma": 1}'

In [ ]:
dm = DataModel.from_dict(json.loads(data_model_json, cls=NumpyDecoder))

INFO:root:{'d': 100, 'normalize_matrices': False, 'data_model_factory_name': 'k_features_factory', 'factory_kwargs': {'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, 'name': '', 'description': '', 'gamma': 1, 'data_model_factory': <function k_features_factory at 0x106bca700>}


In [ ]:
dm

DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='', description='', gamma=1)

In [ ]:
dm.generate_model_matrices()

INFO:root:Initializing data model
DEBUG:root:[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:d: 100
INFO:root:normalize_matrices: False
INFO:root:Ratio of first to last eigenvalue of Σ_x: 1.0
INFO:root:Norm Σ_x: 100.0
INFO:root:Norm Σ_ω: 100.0
INFO:root:Norm Σ_δ: 100.0
INFO:root:Norm Σ_ν: 100.0
INFO:root:Σ_x: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ω: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_δ: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ν: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 

In [ ]:
experiment = Experiment(
    state_evolution_repetitions=1,
    erm_repetitions=1,
    alphas=np.linspace(0,10,5),
    epsilons=np.array([0, 0.1]),
    lambdas=np.array([0.1]),
    taus=np.array([0.1]),
    d=d,
    experiment_type=ExperimentType.Sweep,
    data_model = data_model,
    test_against_epsilons=np.array([0,0.1]),
    erm_problem_type=ProblemType.Logistic,
    gamma_fair_error=0.01,
    name="vanilla-gaussian"
)
experiment_json = experiment.to_json()

In [ ]:
with open("experiment.json", "w") as f:
    f.write(experiment_json)

In [ ]:
e = Experiment.fromdict(json.loads(experiment_json, cls=NumpyDecoder))

INFO:root:{'d': 100, 'normalize_matrices': False, 'data_model_factory_name': 'k_features_factory', 'factory_kwargs': {'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, 'name': '', 'description': '', 'gamma': 1, 'data_model_factory': <function k_features_factory at 0x106bca700>}


In [ ]:
e.data_model

DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='', description='', gamma=1)

In [ ]:
command = ["pwd"]
subprocess.run(command)

/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS


CompletedProcess(args=['pwd'], returncode=0)

In [ ]:
venv_python = ".venv/bin/python"
command = ["mpiexec", "-n", "5", venv_python, "sweep/run_sweep.py", "--json", experiment_json, "--log-level", "DEBUG"]
subprocess.run(command)

2025-02-13 23:01:43,359 - Rank 0 - INFO - The MPI comm size is 5
2025-02-13 23:01:43,359 - Rank 0 - INFO - This process has rank 0
2025-02-13 23:01:43,359 - Rank 0 - INFO - Starting Master for experiment vanilla-gaussian
2025-02-13 23:01:43,359 - Rank 0 - INFO - Starting Experiment vanilla-gaussian
2025-02-13 23:01:43,368 - Rank 0 - INFO - Starting all processes
2025-02-13 23:01:43,368 - Rank 0 - INFO - Sending task 0 to 1
2025-02-13 23:01:43,369 - Rank 0 - INFO - Sending task 1 to 2
2025-02-13 23:01:43,369 - Rank 0 - INFO - Sending task 2 to 3
2025-02-13 23:01:43,370 - Rank 0 - INFO - Sending task 3 to 4
2025-02-13 23:01:43,370 - Rank 0 - INFO - All processes started - receiving results and sending new tasks
2025-02-13 23:01:43,370 - Rank 0 - INFO - Received the 0th task


INFO:root:The MPI comm size is 5
INFO:root:This process has rank 0
INFO:root:Starting Master for experiment vanilla-gaussian
INFO:root:Starting Experiment vanilla-gaussian
INFO:root:The MPI comm size is 5
INFO:root:This process has rank 3
INFO:root:The MPI comm size is 5
INFO:root:The MPI comm size is 5
INFO:root:The MPI comm size is 5
INFO:root:This process has rank 4
INFO:root:This process has rank 1
INFO:root:This process has rank 2
  0%|          | 0/20 [00:00<?, ?it/s]INFO:root:Starting all processes
INFO:root:Sending task 0 to 1
INFO:root:Sending task 1 to 2
INFO:root:Sending task 2 to 3
INFO:root:Sending task 3 to 4
INFO:root:All processes started - receiving results and sending new tasks
INFO:root:Received the 0th task
INFO:root:Starting task 3
INFO:root:Starting task 1
INFO:root:Starting task 2
INFO:root:Starting task 4
DEBUG:root:DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 10

2025-02-13 23:01:44,339 - Rank 0 - INFO - Received task 4 from 4
2025-02-13 23:01:44,339 - Rank 0 - DEBUG - Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'brentq': Cannot determine Numba type of <class 'builtin_function_or_method'>

File "numerics/proximal.py", line 19:
def evaluate_proximal(V: float, y: float, epsilon_term: float, w: float) -> float:
    <source elided>
    w_prime = w - epsilon_term / y
    z = brentq(-50000000, 50000000, 10e-10, 10e-10, 500, y, V, w_prime)
    ^

During: Pass nopython_type_inference
During: resolving callee type: type(CPUDispatcher(<function evaluate_proximal at 0x162cfb880>))
During: typing of call at /Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/state_evolution/logistic_integrals.py (38)

During: resolving callee type: type(CPUDispatcher(<function evaluate_proximal at 0x162cfb880>))
During: typing of call at /Users/kasimirtanner/Documents/LocalC

DEBUG:numba.core.typeinfer:captured error
Traceback (most recent call last):
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numba/core/typeinfer.py", line 160, in propagate
    constraint(typeinfer)
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numba/core/typeinfer.py", line 566, in __call__
    self.resolve(typeinfer, typevars, fnty)
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numba/core/typeinfer.py", line 589, in resolve
    sig = typeinfer.resolve_call(fnty, pos_args, kw_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/.venv/lib/python3.11/site-packages/numba/core/typeinfer.py", line 1560, in resolve_call
    return self.context.resolve_function_type(fnty, pos_args, kw_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

2025-02-13 23:01:44,669 - Rank 0 - INFO - Received task 2 from 2
2025-02-13 23:01:44,669 - Rank 0 - DEBUG - Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'brentq': Cannot determine Numba type of <class 'builtin_function_or_method'>

File "numerics/proximal.py", line 19:
def evaluate_proximal(V: float, y: float, epsilon_term: float, w: float) -> float:
    <source elided>
    w_prime = w - epsilon_term / y
    z = brentq(-50000000, 50000000, 10e-10, 10e-10, 500, y, V, w_prime)
    ^

During: Pass nopython_type_inference
During: resolving callee type: type(CPUDispatcher(<function evaluate_proximal at 0x13052f880>))
During: typing of call at /Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/state_evolution/logistic_integrals.py (38)

During: resolving callee type: type(CPUDispatcher(<function evaluate_proximal at 0x13052f880>))
During: typing of call at /Users/kasimirtanner/Documents/LocalC

...

2025-02-13 23:01:47,982 - Rank 0 - INFO - Received task 17 from 1
2025-02-13 23:01:47,982 - Rank 0 - DEBUG - 'DataModel' object has no attribute 'generate_data'
2025-02-13 23:01:47,982 - Rank 0 - ERROR - Error Task(id=17, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=7.5, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=AttributeError("'DataModel' object has no attribute 'generate_data'"))
2025-02-13 23:01:47,983 - Rank 0 - ERROR - 'DataModel' object has no attribu

INFO:root:d: 100
INFO:root:normalize_matrices: False
INFO:root:Ratio of first to last eigenvalue of Σ_x: 1.0
INFO:root:Norm Σ_x: 100.0
INFO:root:Norm Σ_ω: 100.0
INFO:root:Norm Σ_δ: 100.0
INFO:root:Norm Σ_ν: 100.0
INFO:root:Σ_x: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ω: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_δ: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_ν: [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
INFO:root:Σ_x value counts: (array([0., 1.]), array([9900,  100]))
INFO:root:Σ_ω value counts: (array([

2025-02-13 23:01:48,224 - Rank 0 - INFO - Received task 19 from 3
2025-02-13 23:01:48,231 - Rank 0 - DEBUG - 'DataModel' object has no attribute 'generate_data'
2025-02-13 23:01:48,238 - Rank 0 - ERROR - Error Task(id=19, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.0, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=AttributeError("'DataModel' object has no attribute 'generate_data'"))
2025-02-13 23:01:48,250 - Rank 0 - ERROR - 'DataModel' object has no attrib

INFO:root:Received task 20 from 2
DEBUG:root:'DataModel' object has no attribute 'generate_data'
ERROR:root:Error Task(id=20, task_type=<TaskType.ERM: 0>, erm_problem_type=<ProblemType.Logistic: 0>, alpha=10.0, epsilon=0.1, test_against_epsilons=[0.0, 0.1], lam=0.1, tau=0.1, d=100, data_model=DataModel(d=100, normalize_matrices=False, data_model_factory=k_features_factory, factory_kwargs={'x_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'θ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ω_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'δ_diagonal': KFeaturesDefinition(diagonal=[[1, 100]]), 'ν_diagonal': KFeaturesDefinition(diagonal=[[1, 100]])}, name='', description='', gamma=1), values={}, gamma_fair_error=0.01, gamma=1, result=AttributeError("'DataModel' object has no attribute 'generate_data'"))
ERROR:root:'DataModel' object has no attribute 'generate_data'
INFO:root:All tasks sent and received
INFO:root:Marking experiment vanilla-gaussian as finished
INFO:root:Ex

CompletedProcess(args=['mpiexec', '-n', '5', '.venv/bin/python', 'sweep/run_sweep.py', '--json', '{"name": "vanilla-gaussian", "state_evolution_repetitions": 1, "erm_repetitions": 1, "alphas": [0.0, 2.5, 5.0, 7.5, 10.0], "epsilons": [0.0, 0.1], "test_against_epsilons": [0.0, 0.1], "lambdas": [0.1], "taus": [0.1], "d": 100, "experiment_type": "Sweep", "data_model": {"d": 100, "normalize_matrices": false, "data_model_factory_name": "k_features_factory", "factory_kwargs": {"x_diagonal": {"diagonal": [[1, 100]]}, "\\u03b8_diagonal": {"diagonal": [[1, 100]]}, "\\u03c9_diagonal": {"diagonal": [[1, 100]]}, "\\u03b4_diagonal": {"diagonal": [[1, 100]]}, "\\u03bd_diagonal": {"diagonal": [[1, 100]]}}, "name": "", "description": "", "gamma": 1}, "erm_problem_type": "Logistic", "gamma_fair_error": 0.01}', '--log-level', 'DEBUG'], returncode=0)